**Q-Learning: Aprendendo as Melhores Ações por Tentativa e Erro**

Imagine que você está tentando ensinar um cachorro a sentar. Você não o ensina com uma lista de instruções. Em vez disso, você espera que ele tente algo (talvez ele sente por acidente), e quando ele faz o que você quer, você o recompensa (com um petisco ou carinho). Com o tempo, ele associa a ação "sentar" com a recompensa.

O Q-Learning funciona de forma semelhante. Um agente (o "cachorro") explora um ambiente (o "mundo"), toma ações, recebe recompensas (ou punições) e observa o novo estado do ambiente. Seu objetivo é aprender uma política (um conjunto de regras de comportamento) que o guie a tomar as melhores ações em cada estado para maximizar a recompensa total a longo prazo.

A "inteligência" do Q-Learning reside em uma tabela chamada Tabela Q (Q-Table).

A Tabela Q (Q-Table)

* É uma tabela que armazena os "valores Q" para cada par (estado, ação).
* Um valor Q (Q-value) representa a qualidade (Q) de tomar uma determinada ação em um determinado estado e, a partir daí, seguir a melhor política possível.
* Quanto maior o valor Q para um (estado, ação), melhor é essa ação naquele estado.

In [1]:
!pip install gymnasium

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gymnasium as gym # Usando gymnasium que é o sucessor do gym
import random
from IPython.display import clear_output # Para limpar a saída no Colab/Jupyter
import time # Para adicionar um pequeno delay na renderização

print("Iniciando a demonstração didática de Q-Learning...\n")

# --- 1. Configuração do Ambiente e Parâmetros ---
print("PASSO 1: Configurando o ambiente Frozen Lake e os parâmetros do Q-Learning.")

# Criando o ambiente Frozen Lake (is_slippery=False para simplificar e garantir a convergência rápida)
# Com is_slippery=True, o ambiente é estocástico (ações podem não levar onde esperado), o que é mais complexo.
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="ansi")
env.reset() # Resetar o ambiente para o estado inicial

# Parâmetros do Q-Learning
learning_rate = 0.9      # Alfa (α): Quão rápido o agente atualiza os valores Q
discount_factor = 0.95   # Gamma (γ): Importância das recompensas futuras
epsilon = 1.0            # Epsilon (ε): Probabilidade de explorar (começa alto)
epsilon_decay_rate = 0.001 # Taxa de decaimento do epsilon por episódio
min_epsilon = 0.01       # Epsilon mínimo para garantir alguma exploração
n_episodes = 2000        # Número total de episódios de treinamento

# Inicialização da Tabela Q
# A tabela Q terá o formato (num_estados, num_acoes)
# Preenchemos com zeros no início, pois o agente não sabe nada.
q_table = np.zeros((env.observation_space.n, env.action_space.n))

print(f"  - Número de Estados: {env.observation_space.n}") # 16 estados (4x4)
print(f"  - Número de Ações: {env.action_space.n}")     # 4 ações (cima, baixo, esquerda, direita)
print(f"  - Tabela Q inicial (primeiras 5 linhas):\n{q_table[:5]}\n")
print("---------------------------------------------------\n")

# --- 2. Treinamento do Agente (Q-Learning Loop) ---
print("PASSO 2: Iniciando o treinamento do agente usando Q-Learning.")
print("  -> O agente explorará o lago congelado, aprendendo os valores Q por tentativa e erro.")

rewards_per_episode = []

for episode in range(n_episodes):
    state, info = env.reset() # Resetar o ambiente a cada novo episódio (voltar ao início)
    done = False              # Flag para indicar se o episódio terminou (chegou ao objetivo ou buraco)
    trunc = False             # Flag para indicar se o episódio foi truncado (limite de passos)
    current_episode_reward = 0

    while not done and not trunc:
        # 1. Escolha da Ação (Epsilon-Greedy Strategy)
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Exploração: Escolhe uma ação aleatória
        else:
            action = np.argmax(q_table[state, :]) # Explotação: Escolhe a ação com o maior Q-value

        # 2. Tomada da Ação e Observação do Novo Estado e Recompensa
        new_state, reward, done, trunc, info = env.step(action)

        # 3. Atualização do Valor Q (Fórmula do Q-Learning)
        # Q(s, a) <- Q(s, a) + α * [r + γ * max(Q(s', a')) - Q(s, a)]
        q_table[state, action] = q_table[state, action] + learning_rate * \
                                 (reward + discount_factor * np.max(q_table[new_state, :]) - q_table[state, action])

        state = new_state # Move para o novo estado
        current_episode_reward += reward

    # Decaimento do Epsilon (reduz a exploração com o tempo)
    epsilon = max(min_epsilon, epsilon - epsilon_decay_rate)

    rewards_per_episode.append(current_episode_reward)

    if (episode + 1) % (n_episodes // 10) == 0:
        print(f"  - Episódio {episode + 1}/{n_episodes} - Epsilon: {epsilon:.2f}")

print("\n  Treinamento concluído!")
print("---------------------------------------------------\n")

# --- 3. Análise dos Resultados do Treinamento ---
print("PASSO 3: Analisando o desempenho do treinamento.")

# Média de recompensas por blocos de episódios (para suavizar a curva)
rewards_per_thousand_episodes = np.split(np.array(rewards_per_episode), n_episodes/1000)
count = 1000
print("  Média de recompensas por bloco de 1000 episódios:")
for r in rewards_per_thousand_episodes:
    print(f"    - Episódios {count-999}-{count}: {sum(r/1000)}")
    count += 1000

plt.figure(figsize=(10, 6))
plt.plot(pd.Series(rewards_per_episode).rolling(window=100).mean()) # Média móvel de 100 episódios
plt.title('Recompensa Média por Episódio (Média Móvel de 100 Episódios)', fontsize=14, weight='bold')
plt.xlabel('Episódio')
plt.ylabel('Recompensa Média')
plt.grid(True)
plt.show()

print("\n  A Tabela Q final (representando a política aprendida, primeiras 5 linhas):\n")
print(q_table[:5])
print("\n  Os valores na tabela Q indicam a 'qualidade' de cada ação em cada estado.")
print("  Valores mais altos significam ações melhores.\n")
print("---------------------------------------------------\n")


# --- 4. Avaliação do Agente Treinado (Teste) ---
print("PASSO 4: Avaliando o agente treinado (sem exploração).")
print("  -> O agente agora usará a política aprendida (apenas explotação).")

n_eval_episodes = 10
total_rewards_eval = 0

for episode in range(n_eval_episodes):
    state, info = env.reset()
    done = False
    trunc = False
    print(f"\n--- Avaliação - Episódio {episode + 1} ---")
    time.sleep(0.5) # Pequeno delay para visualização

    while not done and not trunc:
        clear_output(wait=True) # Limpa a tela no Colab/Jupyter
        print(f"--- Avaliação - Episódio {episode + 1} ---")
        print(env.render()) # Mostra o ambiente
        time.sleep(0.3)

        action = np.argmax(q_table[state, :]) # Escolhe a melhor ação aprendida
        new_state, reward, done, trunc, info = env.step(action)
        state = new_state
        total_rewards_eval += reward

    clear_output(wait=True)
    print(f"--- Avaliação - Episódio {episode + 1} ---")
    print(env.render()) # Renderiza o estado final
    if reward > 0:
        print("🎉 Agente alcançou o objetivo! 🎉")
    else:
        print("😔 Agente caiu em um buraco ou esgotou os passos. 😔")
    time.sleep(1)

print(f"\n\n  Média de recompensas na avaliação: {total_rewards_eval / n_eval_episodes:.2f}")
print("  Se o valor for próximo de 1.0, o agente está alcançando o objetivo consistentemente.")
print("  Se o agente cair em um buraco, a recompensa é 0 (ou negativa dependendo do ambiente).")
print("\n--- Demonstração de Q-Learning Concluída! ---")

# Fechar o ambiente
env.close()

--- Avaliação - Episódio 10 ---
  (Right)
SFFF
FHFH
FFFH
HFFG

🎉 Agente alcançou o objetivo! 🎉


  Média de recompensas na avaliação: 1.00
  Se o valor for próximo de 1.0, o agente está alcançando o objetivo consistentemente.
  Se o agente cair em um buraco, a recompensa é 0 (ou negativa dependendo do ambiente).

--- Demonstração de Q-Learning Concluída! ---
